In [1]:
import json
import os
import shutil
import sqlite3
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret

from logzero import logger
from requests import get
import calendar
import regex as re
import pandas as pd

In [2]:
with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [3]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

In [9]:
def db_populate():
    path = cfg["policy_data_dir"]
    
    empty_returns = []
    conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
    cursor_proj = conn_proj.cursor()
    
    for root, dirs, files in os.walk(path):
        for filename in files:
            if ".csv" in filename:
                #print(filename)
                try:
                    d_f = pd.read_csv(path + filename)
                    d_f.to_sql(filename[:-4], conn_proj, if_exists="replace", index=False)
                    conn_proj.commit
                except Exception as e: 
                    empty_returns.append(filename)
    
    """issue_ids = {}
    for eia_id in empty_returns:
        cursor_proj.execute(query.fetch_eia_proj_info, {"eia_id_prob": eia_id})
        eia_info = cursor_proj.fetchall()
        issue_ids[eia_id] = eia_info
    
    logger.info(
        "EIA query errors formatted(site: proj_cap, tbn_state, proj_year)"
        + str(issue_ids)
    )
    logger.info("\n")"""
    conn_proj.close
    return empty_returns

In [10]:
empty_returns = db_populate()